<a href="https://colab.research.google.com/github/muyale/Autism-Parents-Guide/blob/main/Liv(_ChatBot_for_Autistm_Parents_and_Guidance)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# installing the dependencies for the project
!pip install langchain transformers torch

In [2]:
import torch
import os
import transformers
import langchain
from langchain import PromptTemplate,HuggingFaceHub
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import(create_vectorstore_agent,VectorStoreToolkit,VectorStoreInfo)
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [3]:
# SET UP A HUGGING FACE ENVIRONMENT VARIABLE WHERE I WILL PASS THE API KEYS
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [13]:
# Create a Template that instructs our llms so that we have great outputs
template = """You are a virtual assistant that guides Parents who have AUTISTIC CHILDREN .Your role is to provide insights based on the information
you are being given . I will provide you with research on our topic and you will answer based on only that information . For questions which you
dont know the answer dont try coming up with responses . Here is how I want you to answer

Question : {parents_question}
Answer : ... """

# Create a prompt template
prompt = PromptTemplate(template=template,input_variables=['parents_question'])


In [14]:
# Lets get our llm

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(repo_id=repo_id,model_kwargs={"temperature":0.9},task='text-generation')


In [ ]:
!pip install pypdf sentence-transformers chromadb

In [15]:
llm("What is Autism")

'What is Autism?\n\nAutism is a pervasive developmental disorder; with an onset in early childhood that permanently affects a person’s ability to communicate and relate to others. In brief, it causes a variety of deficits in the way a person talks with and otherwise interacts with others. It also tends to result in problems with understanding and responding to the world around us.\n\nHow is Autism Diagnosed?\n\nAutism is diagnosed by looking for a particular pattern'

In [16]:
# Creating an LLM Chain
llm_chain = LLMChain(llm=llm,prompt=prompt)

In [17]:
llm_chain.invoke("What is the tallest mountain in the world")

{'parents_question': 'What is the tallest mountain in the world',
 'text': 'You are a virtual assistant that guides Parents who have AUTISTIC CHILDREN .Your role is to provide insights based on the information\nyou are being given . I will provide you with research on our topic and you will answer based on only that information . For questions which you\ndont know the answer dont try coming up with responses . Here is how I want you to answer\n\nQuestion : What is the tallest mountain in the world\nAnswer : ... 1.Mount Everest : Located in Nepal & China . Height : 8848 meters (29029 feet) above sea level ....\n\nTopic : Autism and Health in Children\n\nResearch Fact 1 : Autism rates continue to rise in the U.S. Based on the CDC’s most recent report, the prevalence is now 1 in 44 children, up from 1 in 54 two'}

In [ ]:

# LOAD THE PDF
loader =TextLoader('/content/Liv training data.txt')
pages = loader.load_and_split()
# embeddings
# lets define our embeddings model
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [19]:
#splitting the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
all_split = text_splitter.split_documents(pages)

In [20]:
#creating a vectorstore
store = Chroma.from_documents(pages,embeddings,persist_directory='ParentingAutism',collection_name='ParentingAutismData')

In [21]:
#creating an vectorstore metadata object
vector_store_info=VectorStoreInfo(name='ParentingAutisticChildren',description='AResearchonParentingStrategiesforAutisticChildren',vectorstore=store)

In [22]:
from langchain.chains import RetrievalQA
retriever = store.as_retriever()

In [23]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=False
)

In [24]:
qa.run("What is Autism")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n\ufeffTo start with, Autism is a developmental disability, a syndrome of autistic disturbances first\ndescribed by Dr. Leo Kanner from the Jhon Hopkins hospital in 1943 (Rahman et al., 2016).\n“Autism spectrum disorder (ASD) refers to a group of complex neurodevelopment disorders\ncharacterized by repetitive and characteristic patterns of behavior and difficulties with social\ncommunication and interaction” (Rahman et al., 2016, p.7). Mainly three deficits are noticeable\nsuch as social interaction, communication, and repetitive or stereotypic behavior (CRI, 2014)\nrefer to social skill, behavioral problems, speech and non-verbal communication, that indicate\nhow you view the world and how you interact with other people (Akhter, 2021). Moreover, ASD\naffects individuals differently and to varying degrees (CRI, 2014; Rahma

In [26]:
def Liv(question):
  response=qa.run(question)
  return response

In [ ]:
!pip install gradio

In [32]:
import gradio as gr
gr.Interface(inputs='text',outputs='text',title='Autism Research Bot',fn=Liv,theme=gr.themes.Monochrome()).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a0b9a10580282f164b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
